In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.datasets import EllipticBitcoinDataset

from ogb.nodeproppred import PygNodePropPredDataset

In [4]:
elliptic = EllipticBitcoinDataset(root='./data/Elliptic/')
ogbn_arxiv = PygNodePropPredDataset(name = "ogbn-arxiv", root = './data/Ogbn/')

Downloaded 0.08 GB: 100%|███████████████████████████████████████████████████████████| 81/81 [01:04<00:00,  1.25it/s]


Extracting ./data/Ogbn/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Converting graphs into PyG objects...


100%|███████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1001.98it/s]

Saving...



Done!


In [5]:
print(len(elliptic), len(ogbn_arxiv))

1 1


In [7]:
print(elliptic.num_classes, ogbn_arxiv.num_classes)
print(elliptic.num_node_features, ogbn_arxiv.num_node_features)

2 40
165 128


In [11]:
from torch_geometric.utils import homophily
edge_index = elliptic.edge_index
y = elliptic.y
homophily(edge_index, y, method='edge')

0.7113396525382996

In [12]:
edge_index = ogbn_arxiv.edge_index
y = ogbn_arxiv.y
homophily(edge_index, y, method='edge')

0.6550830602645874

In [13]:
from torch_geometric.data import InMemoryDataset
import numpy as np

class CustomGraphDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(CustomGraphDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['adjlist.npy', 't.npy', 'X.npy', 'y.npy']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        adjlist = np.load(self.raw_paths[0])
        t = np.load(self.raw_paths[1])
        X = np.load(self.raw_paths[2])
        y = np.load(self.raw_paths[3])

        edge_index = torch.tensor(np.vstack(np.where(adjlist)).astype(np.int64), dtype=torch.long)
        x = torch.tensor(X, dtype=torch.float)
        y = torch.tensor(y, dtype=torch.float).view(-1, 1)

        # Create a PyTorch Geometric Data object
        data = Data(x=x, edge_index=edge_index, y=y)

        # Add other properties as needed

        # Save the dataset
        torch.save((data, self.slices), self.processed_paths[0])

In [14]:
dblp_hard = CustomGraphDataset(root='./../data/dblp-hard-citation-graph/dblp-hard/')

Processing...


FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\dblp-hard-citation-graph\\dblp-hard\\raw\\adjlist.npy'